# Context Managers

Context managers are normally used to allocate and release resources automatically without needing to worry about about doing it yourself. You normally use the `with` keyword when dealing with CMs.

The most common example of a CM is when working with file IO. You normally see something like:
```python
with open('somefile.txt') as f:
    do something...
```
We show an example below.

In [4]:
with open('static/Genesis.txt', 'r') as f:
    print(next(f))
    print(next(f))

[1:1] In the beginning when God created the heavens and the earth,

[1:2] the earth was a formless void and darkness covered the face of the deep, while a wind from God swept over the face of the waters.



Here we open the file, print the first two lines, and then closes the file. The `open` CM handles opening and closing the file for you. The file is opened when you enter the `with` block and is closed when exiting the block. You can see that this is cleaner then manually opening and closing the file like below. Which is also prone to errors like forgetting to close the file yourself.

In [5]:
f = open('static/Genesis.txt', 'r')
print(next(f))
print(next(f))
f.close()

[1:1] In the beginning when God created the heavens and the earth,

[1:2] the earth was a formless void and darkness covered the face of the deep, while a wind from God swept over the face of the waters.



In general, CMs are used to automatically run some sort of setup and then teardown code.

We can thus leverage our own CMs to automatically run some "setup" and "teardown" to do any number of tasks.

There are two main ways to create a Context Manager:
1. By defining a class.
1. By defining a generator.

***



In [1]:
from time import time

class Timer:
    def __enter__(self):
        self.start = time()
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.end = time()
        print(self.end - self.start, 'seconds to run')

In [33]:
with Timer():
    res = 0
    for i in range(10_000_000):
        res += i

1.3250179290771484 seconds to run


In [34]:
from contextlib import contextmanager

@contextmanager
def timer():
    start = time()
    yield
    end = time()
    print(end - start, 'seconds to run')

In [35]:
with timer():
    res = 0
    for i in range(10_000_000):
        res += i

1.3074147701263428 seconds to run
